In [1]:
#!pip install deeppavlov spacy

In [2]:
#!python -m spacy download en

In [3]:
import pandas as pd
import spacy
from spacy import displacy
from collections import Counter

In [4]:
nlp = spacy.load("en")

In [5]:
combine_df = pd.read_pickle("../Lesson_01/data/combine_df.zip")

In [6]:
combine_df.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [7]:
cntr = Counter()
ents_dict = {}

In [8]:
def parse_string_entities(text_string, cntr, ents_dict):
    doc = nlp(text_string)
    ents = [(e.text, e.label_) for e in doc.ents]
    for entity in ents:
#         print("text:", entity[0])
#         print("label:", entity[1])
        cntr[entity[0]] += 1
        if entity[0] not in ents_dict:
            ents_dict[entity[0]] = entity[1]
        else:
            if ents_dict[entity[0]] != entity[1]:
                print("ENTITIES MISMATCH:\n\tentity text: %s\n\tdict label: %s\n\tnew label: %s\n" % (entity[0], ents_dict[entity[0]], entity[1]))
                

In [9]:
# strstr = "I think Barack Obama met founder of Facebook at occasion of a release of a new NLP algorithm."
# parse_string_entities(strstr,cntr, ents_dict)

In [10]:
%%time
for i in range(len(combine_df)):
#     parse_string_entities(combine_df.iloc[i]['tweet'], cntr, ents_dict)
    parse_string_entities(combine_df.iloc[i]['clean_tweet'], cntr, ents_dict)

ENTITIES MISMATCH:
	entity text: early tomorrow
	dict label: DATE
	new label: TIME

ENTITIES MISMATCH:
	entity text: early tomorrow
	dict label: DATE
	new label: TIME

ENTITIES MISMATCH:
	entity text: sixteen
	dict label: CARDINAL
	new label: DATE

ENTITIES MISMATCH:
	entity text: sativa
	dict label: CARDINAL
	new label: DATE

ENTITIES MISMATCH:
	entity text: early sunday
	dict label: TIME
	new label: DATE

CPU times: user 3min 50s, sys: 229 ms, total: 3min 50s
Wall time: 3min 50s


In [11]:
cntr.most_common(20)

[('today', 1620),
 ('summer', 652),
 ('friday', 643),
 ('tomorrow', 622),
 ('first', 593),
 ('day', 558),
 ('sunday', 469),
 ('one', 455),
 ('tonight', 448),
 ('fathersday', 403),
 ('morning', 353),
 ('weekend', 314),
 ('saturday', 301),
 ('days', 297),
 ('monday', 252),
 ('two', 189),
 ('years', 163),
 ('the day', 162),
 ('thursday', 157),
 ('week', 151)]

In [12]:
for common in cntr.most_common(5):
    word = common[0]
    ent_label = ents_dict[word]
    print(word, ent_label)

today DATE
summer DATE
friday DATE
tomorrow DATE
first ORDINAL


In [13]:
%%time
tmp_cntr = 0
popular_orgs_and_persons = []
for i, common in enumerate(cntr.most_common(10000)):
    word = common[0]
    count = common[1]
    ent_label = ents_dict[word]
    if ent_label == "PERSON" or ent_label == "ORG":
        popular_orgs_and_persons.append((word, ent_label, count))
    else:
        print(i, word, ent_label, count)
    if len(popular_orgs_and_persons) == 20:
        break

0 today DATE 1620
1 summer DATE 652
2 friday DATE 643
3 tomorrow DATE 622
4 first ORDINAL 593
5 day DATE 558
6 sunday DATE 469
7 one CARDINAL 455
8 tonight TIME 448
9 fathersday DATE 403
10 morning TIME 353
11 weekend DATE 314
12 saturday DATE 301
13 days DATE 297
14 monday DATE 252
15 two CARDINAL 189
16 years DATE 163
17 the day DATE 162
18 thursday DATE 157
19 week DATE 151
20 wednesday DATE 135
21 yesterday DATE 131
22 daily DATE 113
23 this week DATE 112
24 next week DATE 111
25 this morning TIME 107
26 weeks DATE 103
27 this weekend DATE 102
28 tuesday DATE 96
29 last night TIME 95
30 this year DATE 95
31 year DATE 89
32 night TIME 82
33 hours TIME 79
34 half CARDINAL 70
35 american NORP 69
36 evening TIME 68
37 the weekend DATE 64
38 months DATE 62
39 father day DATE 60
40 month DATE 57
41 second ORDINAL 55
42 last day DATE 54
43 season DATE 54
44 afternoon TIME 53
45 minutes TIME 49
46 three CARDINAL 45
47 humpday DATE 44
48 next year DATE 41
49 this summer DATE 40
50 the days 

1349 september school end summer day DATE 1
1350 fadaz day DATE 1
1351 wednesday mid week DATE 1
1352 luzern hour TIME 1
1353 shbeen CARDINAL 1
1354 many years ago DATE 1
1355 is hour TIME 1
1356 awesome weekend DATE 1
1357 rainyday night TIME 1
1358 wednesdday DATE 1
1359 truth day DATE 1
1360 the this weekend DATE 1
1361 monday tuesday wednesday DATE 1
1362 songoftheday DATE 1
1363 old days DATE 1
1364 the next one DATE 1
1365 feel third year DATE 1
1366 natinalbestfriendday TIME 1
1367 and few hours TIME 1
1368 photoftheday DATE 1
1369 terrific week DATE 1
1370 one year ago DATE 1
1371 about years DATE 1
1372 to day DATE 1
1373 just one CARDINAL 1
1374 less than ten minutes TIME 1
1375 day tomorrow DATE 1
1376 the weekend oscar friday DATE 1
1377 autoimmunewarrior day DATE 1
1378 like minutes TIME 1
1379 crappy day DATE 1
1380 thursday mixed day DATE 1
1381 mirrormonday DATE 1
1382 happydays day DATE 1
1383 st months DATE 1
1384 this summer summer DATE 1
1385 election year DATE 1
13

In [14]:
popular_orgs_and_persons

[('msnbc', 'ORG', 14)]

### spacy очень плохо отловил лэйблы, которые относятся к искомым категориям (PERSON, ORG, ..). Насколько я понял это связано с тем что весь текст в clean_tweet у нас lower.

In [16]:
text_string = "Barak Obama was a first black skinned president of USA"
doc = nlp(text_string)
ents = [(e.text, e.label_) for e in doc.ents]
ents

[('Barak Obama', 'PERSON'), ('first', 'ORDINAL'), ('USA', 'GPE')]

In [17]:
text_string = "Barak Obama was a first black skinned president of USA".lower()
doc = nlp(text_string)
ents = [(e.text, e.label_) for e in doc.ents]
ents

[('first', 'ORDINAL')]